
# **EMOTIONAL SPEECH RECOGNITION ANALYSIS**

## English
El siguiente analisis presenta una clasificacion emocional del discurso, reconociendo 7 emociones (enfado, sorpresa, asco, miedo, felicidad, tristeza y neutral) para el idioma **ingles**.

Luisa Sanchez Avivar
    _luisasanavi@gmail.com_

In [1]:
# IMPORT LIBRARIES
# Processing
import librosa
import librosa.display
import numpy as np
import random
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Files
import os

# Machine Learning
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
import keras
from keras.utils import np_utils, to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D

# ####### TEST ####### 
# Scipy
from scipy import signal
from scipy.io import wavfile



In [2]:
AUDIO_DATA_PATH = 'data/'
GPATH = '/content/drive/My Drive/Master/Asignaturas/2 Cuatrimestre/Proyecto/Code/'
SAMPLE_FILE = "03-01-01-01-01-01-01.wav"

# Maps
EMOTION_MAP = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
INTENSITY_MAP = {1:'normal', 2:'strong'}


## **1. CARGA DE DATOS**
Para este analisis utilizare el dataset de [RAVDESS](https://zenodo.org/record/1188976.) (Ryerson Audio-Visual Database of Emotional Speech and Song), el cual contiene 7356 archivos (24.8 GB) entre los cuales podemos encontrar 3 modalidades: **solo audio** (en 16 bit, 48 kHz y en formato .wav), **audio-video** (720p H.264, AAC 48kHz, .mp4) y **solo video** sin sonido.





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir_list = os.listdir(GPATH + AUDIO_DATA_PATH)
dir_list.sort()

print(dir_list)

emotion = []
gender = []
intensity = []
path = []

# Extraemos de cada archivo de sonido sus datos
for dir in dir_list:
  path_dir = os.listdir(GPATH + AUDIO_DATA_PATH + dir) # todos los archivos de audios asociados a un directorio
  for filepath in path_dir:
    info_vector = filepath.split('.')[0].split('-')
    n_emotion = int(info_vector[2])
    n_gender = int(info_vector[6])
    n_intensity = int(info_vector[3])
    str_path = GPATH + AUDIO_DATA_PATH + dir + '/' + str(filepath)
    path.append(str_path)
    emotion.append(n_emotion)
    intensity.append(n_intensity)
    if n_gender%2 == 0:
      gender.append('female')
    else:
      gender.append('male')

# Construimos el data frame
EnglishSpeech_df = pd.DataFrame(columns=['emotion', 'gender', 'intensity', 'path'])
EnglishSpeech_df['emotion'] = emotion
EnglishSpeech_df['gender'] = gender
EnglishSpeech_df['intensity'] = intensity
EnglishSpeech_df['path'] = path
EnglishSpeech_df['emotion'] = EnglishSpeech_df['emotion'].map(EMOTION_MAP) 
EnglishSpeech_df['intensity'] = EnglishSpeech_df['intensity'].map(INTENSITY_MAP)


print("Size of the dataset: {} \n".format(len(EnglishSpeech_df)))
class_distribution = EnglishSpeech_df['emotion'].value_counts()
print(class_distribution)

In [ ]:
# Imprimimos la distribucion de las clases

df_aux = pd.DataFrame()
df_aux['emotion'] = list(class_distribution.keys())
df_aux['count']  = list(class_distribution)
fig, axs = plt.subplots(figsize=(15, 5))
axs = sns.barplot(x = 'emotion', y = 'count', color = '#2962FF', data = df_aux)
axs.set_title('Distribucion')
axs.set_xticklabels(axs.get_xticklabels(),rotation=45, fontsize = 12)
plt.show()

In [ ]:
# Imprimimos una muestra de 10 filas aleatorias
EnglishSpeech_df.sample(n = 10)

# **2. EXPLORACION DE LOS DATOS**

### **2.1 EXPOSICION DE UNA MUESTRA ALEATORIA**



In [ ]:

def plot_audio_waveform(audio_sample):
  '''
  Muestra la forma de la onda sonora a partir de una muestra.

  Arguments
  ---------
  audio_sample: dataFrame
    Muestra de audio 
  
  '''
  sample, sampling_rate = librosa.load(audio_sample)
  plt.figure(figsize=(12, 4))
  librosa.display.waveplot(sample, sr=sampling_rate)
  print(len(sample))
  

def log_specgram(audio_sample):
  '''
  Muestra el espectograma a partir de una muestra de audio

  Arguments
  ---------
  audio_sample: dataFrame
    Muestra de audio
  '''
  sample, sampling_rate = librosa.load(audio_sample)
  return  __log_specgram(sample, sampling_rate)


def __log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
  '''
  Imprime el especograma de una muestra de audio en una ventana de tiempo
  
  Arguments
  ---------
  audio: np.ndarray
    Muestra de audio
  sample_rate: int, optional
    Frecuencia de muestreo de la muestra de audio
  window_size: int, optional

  step_size: int, optional
  
  eps: int, optional
  '''
  nperseg = int(round(window_size * sample_rate / 1e3))
  noverlap = int(round(step_size * sample_rate / 1e3))
  freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
  return freqs, times, np.log(spec.T.astype(np.float32) + eps)


In [ ]:
random_sample = EnglishSpeech_df.path[random.randint(0, len(EnglishSpeech_df))]
print("Random audio sample: {}".format(random_sample))
# Dibuja la grafica de la onda sonora
plot_audio_waveform(random_sample)

# Dibuja el espectograma
freqs, times, spectrogram = log_specgram(random_sample)

# Muestra la firgura
plt.figure(figsize=(30, 2))
plt.imshow(spectrogram)
plt.xlabel('Seg')
plt.ylabel('Hz')

In [ ]:

# samplingFrequency, signalData = wavfile.read(random_sample)
# plt.plot(sample)
# plt.specgram(sample, Fs=sampling_rate)

### **2.2 COMPARATIVA DE TODAS LAS EMOCIONES**
Ahora que hemos conseguido extraer la grafica de una determinada muestra de sonido, vamos a comparar la graficas correspondientes a las muestras de sonido de distintas emociones respectivamente. Para ello vamos a distinguir por genero, ya que podria suponer una diferencia.

In [ ]:
def plot_all_emotion_waveforms(gender, rows = 3, cols = 3):
  '''
  Muestra los graficos para todas las emociones con su correspondiente 
  etiqueta del dataset para un mismo genero (female/ male)
  
  Arguments
  --------- 
    gender: str
      Genero del actor en el audio
    rows:  int, optional
      Filas en las que se muestra. 5 por defecto
    cols: int, optional
      Columnas en las que se muestra. 2 por defecto
    
  '''
  labels = list(EnglishSpeech_df['emotion'].unique())
  files = dict()
  if not gender:
    return -1

  # Seleccionamos una muestra aleatoria correspondiente a cada emocion
  for label in labels:
    # Escogemos un archivo de audio al azar que cumpla estas dos condiciones
    index = EnglishSpeech_df[(EnglishSpeech_df['emotion'] == label) & 
                            (EnglishSpeech_df['gender'] == gender)].sample(n = 1).index[0]
    emotion_file = EnglishSpeech_df.iloc[index].path
    files[label] = emotion_file

  # Mostramos las diferentes waveforms
  fig = plt.figure(figsize=(15,15))
  fig.subplots_adjust(hspace=1, wspace=0.4)
  for i, label in enumerate(labels):
    wfigure = files[label]
    fig.add_subplot(rows, cols, i+1)
    plt.title(label.capitalize())
    data_sample, sample_rate = librosa.load(wfigure)
    librosa.display.waveplot(data_sample, sr= sample_rate)

    ## TODO: return image figure


### 2.2.1 COMPARATIVA DE EMOCIONES PARA HOMBRE
En primer lugar vemos que aspecto tienen las emociones para la **voz masculina**

In [ ]:
plot_all_emotion_waveforms('male')


### 2.2.2 COMPARATIVA DE EMOCIONES DE MUJER
Vemos el aspecto que tienen las emociones para la **voz femenina**


In [ ]:
plot_all_emotion_waveforms('female')

Se puede observar en los resultados, que la se;al asociada a cada una de las emociones presenta diferencias con respecto a la voz femenina y la masculina. A simple vista no podemos hablar de unos cambios constantes en determinados puntos. Vemos que la *felicidad* masculina y femennina es expresada de una manera similar, al contrario, por ejemplo, de lo que ocurre con el *miedo*, donde en su version masculina presenta mas saltos entre sus picos de frecuencia. por lo que es algo que deberemos tener en cuenta en este analisis

# **3. EXTRACCION DE CARACTERISTICAS**

### **3.1 EXTRACCION DE CARACTERISTICAS CON MFCC**

In [ ]:
def get_features(df):
  '''
  Extrae las caracteristicas de un conjunto de pistas de audio a 
  partir de un dataframe usando librosa

  Aguments
  ---------
    df : dataframe
    Dataframe que contiene el path donde se encuentra la pista de audio

  Return
  -------
   data: np.array 
   Caracteristicas extraidas

  '''
  bar_data_range = tqdm(range(len(df)))
  data = pd.DataFrame(columns = ['data'])
  for index in bar_data_range:
    data_features = get_features_single_file(df.path[index])
    data.loc[index] = [data_features]

  return data


def get_features_single_file(pathfile):
  '''
  Extrae las caracteristicas  de una unica pista de audio usando MFCC 
  a traves de librosa.
  
  Aguments
  ---------
    pathfile: str 
      Path del archivo del que se extraeran las caracteristicas

  Return
  -------
    data_features

  '''
  X, sample_rate = librosa.load(pathfile, duration=2.5, sr=22050*2, offset=0.5)
  mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)
  data_features = np.mean(mfcc, axis = 0)

  return data_features



def get_random_emotion(df, emotion):
  '''
  Devuelve el path de un archivo de audio aleatorio a partir de un dataframe.

  Aguments
  ---------
  df: dataframe
    Caracteristicas de la muestra de audio organizadas por emociones
  emotion: str
    Nombre de la emocion 
    
  Return
  -------
  '''
  if 'emotion' not in df:
    return -1

  aux_df = df[df['emotion'] == emotion]
  item = random.choice(aux_df.index.to_list())
  path = aux_df.path[item]

  return path

def plot_waves_comparative(df1, df2, df1_title = 'Wave 1', df2_title = 'Wave 2', title_ = 'Title'):
  '''
  Imprime la grafica de dos waveforms a partir de sus caracteristicas.

  Aguments
  ---------
  df1: dataframe
    Caracteristicas de la primera muestra de audio
  df2: dataframe
    Caracteristicas de la segunda muestra de audio
  df1_title: str, optional
    Titulo para la primera grafica
  df2_title: str, optional
    Titulo para la segunda grafica 
  title_: str, optional
    Titulo para la figura

  '''
  plt.figure(figsize=(20, 15))
  plt.subplot(3,1,1)
  plt.title(title_)
  plt.plot(df1, label= df1_title)
  plt.plot(df2, label= df2_title)
  plt.legend()


def plot_all_comparative_waveforms(rows = 3, cols = 3):
  '''


  Aguments
  ---------
  Return
  -------
  '''

  labels = list(EnglishSpeech_df['emotion'].unique())
  features_dict = dict()
  for label in labels:
    # Female
    path = get_random_emotion(EnglishSpeech_df[EnglishSpeech_df['gender'] == 'female'], label)
    # female_feat = get_features_single_file(path)
    key = 'female_' + label
    features_dict[key] = get_features_single_file(path)
    # Male
    path = get_random_emotion(EnglishSpeech_df[EnglishSpeech_df['gender'] == 'male'], label)
    # male_feat = get_features_single_file(path)
    key = 'male_' + label
    features_dict[key] = get_features_single_file(path)

  # Mostramos las diferentes waveforms
  fig = plt.figure(figsize=(15,15))
  fig.subplots_adjust(hspace=0.4, wspace=0.4)
  for i, label in enumerate(labels):
    key = 'female_' + label
    df_fem = features_dict[key]
    key = 'male_' + label
    df_mal = features_dict[key]
    fig.add_subplot(rows, cols, i+1)
    plt.title(label)
    plt.plot(df_fem, label= 'female')
    plt.plot(df_mal, label= 'male')
    plt.legend()



In [ ]:
# Guardamos las caracteristicas a partir de la estructura que hemos construido antes
features_data = get_features(EnglishSpeech_df)
print(features_data)

In [ ]:
# features_data.head()
f_df = pd.DataFrame(features_data['data'].values.tolist())
f_df.head()

In [ ]:
features_complete_df = pd.concat((f_df, EnglishSpeech_df['gender'], EnglishSpeech_df['emotion']), axis = 1)
features_complete_df = features_complete_df.fillna(0)

# Barajamos las filas para imrpimir una muestra 
random_aux = shuffle(features_complete_df)
random_aux.head()

### Division del dataframe
Vamos a dividir el dataframe por el genero, para comprobar como afecta este a la prosodia en la voz femenina y masculina por separadas, dadas las mismas emociones en el analisis

In [ ]:
male_features_df = features_complete_df[(features_complete_df['gender'] == 'male')]
female_features_df = features_complete_df[(features_complete_df['gender'] == 'female')]

### Comparativa
Ahora que podemos extraer las caracteristicas de las señales, vamos a comparar la misma emocion, (por ejemplo *enfado*) en la voz femenina y la masculina.


In [ ]:
# Extraemos una muestra aleatoria para la emocion: enfado 
# Female
path = get_random_emotion(EnglishSpeech_df[EnglishSpeech_df['gender'] == 'female'], 'angry')
female_feat = get_features_single_file(path)
# print(len(female_feat))
# Male
path = get_random_emotion(EnglishSpeech_df[EnglishSpeech_df['gender'] == 'male'], 'angry')
male_feat = get_features_single_file(path)
# print(len(male_feat))

plot_waves_comparative(female_feat, male_feat, "Female", "Male", "angry")



In [ ]:
plot_all_comparative_waveforms()

### **3.2 PREPARACION DE LOS DATOS**


In [ ]:
def split_training_test(df, n_splits_=1, test_size_=0.25, train_size_=None):
  '''
  Divide el dataset en entrenamieto y test utilizando StratifiedShuffleSplit
  Aguments
  ---------
  Return
  -------
  '''
  X = df.drop(['gender', 'emotion'], axis=1)
  Y = df.emotion
  test_train_stratified = StratifiedShuffleSplit(n_splits = n_splits_, test_size = test_size_, random_state=12)
  for train_index, test_index in test_train_stratified.split(X, Y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

  return X_train, X_test, Y_train, Y_test
  # return train_test_split(X, Y, stratify = Y, test_size=0.25)#X_train, X_test, Y_train, Y_test


def data_normalization(val_train, val_test):
  '''
  Normaliza los datos mejorando la precision y la velocidad del proceso de entrenamiento
  Aguments
  ---------
  Return
  -------
  '''
  # Now because we are mixing up a few different data sources, it would be wise to normalise the data. 
  # This is proven to improve the accuracy and speed up the training process. Prior to the discovery of this solution in the embrionic years of neural network, 
  # the problem used to be know as "exploding gradients".
  mean = np.mean(val_train, axis=0)
  std = np.std(val_train, axis=0)

  X_train = (val_train - mean)/std
  X_test = (val_test - mean)/std

  return X_train, X_test


def data_to_categorical(x_train_norm, y_train, x_test_norm, y_test):
  '''
  Categoriza los datos y los formatea para su uso con keras. Asume que x_train y x_test estan normalizados.
  Aguments
  ---------
  Return
  -------
  '''
  # Lets few preparation steps to get it into the correct format for Keras 
  # Preparamos los datos para la categorizacion
  x_train_norm = np.array(x_train_norm)
  y_train = np.array(y_train)
  x_test_norm = np.array(x_test_norm)
  y_test = np.array(y_test)

  # One hot encode 
  label_encoder = LabelEncoder()
  y_train = np_utils.to_categorical(label_encoder.fit_transform(y_train))
  y_test = np_utils.to_categorical(label_encoder.fit_transform(y_test))

  # # Pickel the lb object for future use 
  # filename = 'labels'
  # outfile = open(filename,'wb')
  # pickle.dump(label_encoder, outfile)
  # outfile.close()

  # print(x_train_norm.shape)
  # print(label_encoder.classes_)

  return x_train_norm, y_train, x_test_norm, y_test, label_encoder



### 3.2.1 DIVISION DE LOS DATOS EN ENTRENAMIENTO Y TEST

In [ ]:
# NOTA: Aqui estoy comprobando los datos que me da si divido entrenamiento y test con split de toda la vida

# # female_X_train, female_X_test, female_Y_train, female_Y_test =split_training_test(female_features_df)
# female_X_train.head()
# print(918 in female_X_train.index)

In [ ]:
X_train, X_test, Y_train, Y_test = split_training_test(features_complete_df)
X_train.head()
print(X_train.shape)

In [ ]:
# Comprobamos la distribución de las clases (emociones en nuestro caso)
Y_train.value_counts()

In [ ]:
# Comprobamos que no hay valores NaN
X_train.isna().sum().sum()

### 3.2.2 NORMALIZACION DE LOS DATOS


In [ ]:
X_train_norm, X_test_norm = data_normalization(X_train, X_test)
print(features_complete_df.shape)

# Comprobamos imprimiendo una muestra de los datos
indx = random.randint(0, len(features_complete_df))
X_train[indx:indx+10]

### 3.2.3 CATEGORIZACION DE LOS DATOS

In [ ]:
X_train, Y_train, X_test, Y_test, labels_whole = data_to_categorical(X_train_norm, Y_train, X_test_norm, Y_test)

In [ ]:
print(labels_whole.classes_)

### 3.2.4 CAMBIO DE DIMENSION

In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print(X_train.shape)

Comprobamos que despues de los cambios hay unas dimensiones coherentes

In [ ]:
print("X Train: {} --> Y Train: {}".format(X_train.shape, Y_train.shape))
print("\nX Test: {} --> Y Test: {}".format(X_test.shape, Y_test.shape))

### **3.3 PREPARACION DE LOS DATOS PARA _Female_**

In [ ]:
# 1. Division de los datos: Entrenamiento y Test
female_X_train, female_X_test, female_Y_train, female_Y_test =split_training_test(female_features_df) # with StratifiedShuffleSplit

# 2. Normalizacion
x_train_female_norm, x_test_female_norm = data_normalization(female_X_train, female_X_test)
print(x_train_female_norm.shape)

# 3. Categorización
female_X_train, female_Y_train, female_X_test, female_Y_test, labels_female = data_to_categorical(x_train_female_norm, female_Y_train, x_test_female_norm, female_Y_test)
print(labels_female.classes_)

# 4. Cambio de Dimensión
X_train_female = np.expand_dims(female_X_train, axis=2)
X_test_female = np.expand_dims(female_X_test, axis=2)

print("\nTrain size: {}\nTest size:{}".format(X_train_female.shape, X_test_female.shape))
print("\nTrain size: {}\nTest size:{}".format(female_Y_train.shape, female_Y_test.shape))

### **3.4 PREPARACION DE LOS DATOS PARA _Male_**


In [ ]:
# 1. Division de los datos: Entrenamiento y Test
male_X_train, male_X_test, male_Y_train, male_Y_test =split_training_test(male_features_df)
male_X_train.head()

# 2. Normalizacion
x_train_male_norm, x_test_male_norm = data_normalization(male_X_train, male_X_test)
print(x_train_male_norm.shape)

# 3. Categorización
male_X_train, male_X_test, male_Y_train, male_Y_test, labels_male = data_to_categorical(x_train_male_norm, male_Y_train, x_test_male_norm, male_Y_test)
print(labels_male.classes_)

# 4. Cambio de Dimensión
X_train_male = np.expand_dims(X_train, axis=2)
X_test_male = np.expand_dims(X_test, axis=2)

print("\nTrain size: {}\nTest size:{}".format(X_train_female.shape, X_test_female.shape))

# **4. EL MODELO**
## 4.1 Diseño del modelo

In [ ]:
def model_cnn(x_train, n_classes):
  model = Sequential()

  model.add(Conv1D(128, 5,padding='same',
                  input_shape=(X_train.shape[1],1) ))
  model.add(Activation('relu'))
  model.add(Dropout(0.1))
  model.add(MaxPooling1D(pool_size=(8)))
  model.add(Conv1D(128, 5,padding='same',))
  model.add(Activation('relu'))
  model.add(Dropout(0.1))
  model.add(Flatten())
  # Numero de clases
  model.add(Dense(n_classes))
  model.add(Activation('softmax'))

  return model


## Compilación del modelo

In [ ]:
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# X_train_female , X_test_female
model = model_cnn(X_train_female, female_Y_test.shape[1])
model.summary()


In [ ]:
# opt = keras.optimizers.Adam(lr=0.0001)
# opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
#opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='categorical_crossentropy', 
              optimizer = opt, 
              metrics=['accuracy'])


# Callbacks para tratar el overfitting
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.000001)
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 15)

history = model.fit(X_train_female, female_Y_train, 
                        batch_size=16, 
                        epochs=100, 
                        validation_data=(X_test_female, female_Y_test),
                        callbacks=[rlrp])

### Persistencia del modelo
Guardamos el modelo para un uso posterior sin necesidad de volverlo a entrenar


In [ ]:
# detectar que modelo acaba de ejecutarse para guardarlo automaticamente?
model.save('model_english_female_SER.h5')

## Validación del modelo

In [ ]:

# Mostramos la grafica loss 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(history.history['loss'])
ax1.plot(history.history['val_loss'])
ax1.set_title('Loss')
ax1.set(xlabel='epoch', ylabel='loss')
ax1.legend(['train', 'test'], loc='upper right')

# Mostramos la grafica accuracy
ax2.plot(history.history['accuracy'])
ax2.plot(history.history['val_accuracy'])
ax2.set_title('Validation')
ax2.set(xlabel='epoch', ylabel='acc')

ax2.legend(['train', 'test'], loc='lower right')

In [ ]:
# Evaluamos contra test
score = model.evaluate(X_test_female, female_Y_test, batch_size=128)
print("Loss: {} \nAccuracy: {}%".format(score[0], score[1]*100))

## Predicción

In [ ]:
# Predecimos los valores a partir de los datos de test
y_pred = model.predict(X_test_female)
y_pred_class = np.argmax(y_pred, axis = 1)
y_true = np.argmax(female_Y_test, axis = 1)

In [ ]:
# labels_female
y_pred_class.astype(int).flatten()

In [ ]:
y_true

In [ ]:
cm = confusion_matrix(y_true, y_pred_class)
sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()